# Capstone Project  
## The Battle of Neighborhoods (Week 1)
#####  2. A description of the data and how it will be used to solve the problem. (15 Marks)
##### by Ramakrishna Samudrala
###### 2. Data

###### Uploading all the required packages for this project - 

In [3]:
import numpy as np # This is the library to handle data in a vectorized manner
import time
import pandas as pd # This library is for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # This library is to handle JSON files
import requests # This library is to handle all requests
from pandas.io.json import json_normalize # This is to tranform JSON file into a pandas dataframe


!conda install -c conda-forge geopy   
from geopy.geocoders import Nominatim # This is to convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0   
import folium # This is a map rendering library
import folium # This is a map rendering library
from folium import plugins

# This is for Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns

# This is to import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


#### Capstone Project - Contents

1. Introduction - Discussing and introducing the the business problem to the audience who would be interested in this project.
2. Data - Description of the data and its sources that will be used to solve the problem
3. Methodology - In this methodology section, I will be discussing the main component of the report and describing the exploratory data analysis that was being carried out, any inferential statistical testing performed, and if any machine learnings were used establishing the strategy and purposes.
4. Results - Discussing the results.
5. Discussion - Elaboration and discussion on any observations noted and any recommendations suggested based on the results.
6. Conclusion - Summarizing the Project with the Conclusion.

In [5]:
#  A description of the data and how it will be used to solve the problem. (15 Marks)

2. Data:
Description of the data and its sources that will be used to solve the problem

Data for Present Situation (current residence place)
Currently, my friend is living in the neighborhood of 'Valachery' in Chennai, India. 
I use Foursquare to identify the venues around the area of his residence which are then shown in the Chennai map. 
It serves as a reference for comparison with the desired future location in Manhattan NY

The following data is required to answer the issues of the problem:

List of Boroughs and neighborhoods of Manhattan with their geodata (latitud and longitud)
List of Subway metro stations in Manhattan with their address location
List of apartments for rent in Manhattan area with their addresses and price
Preferably, a list of apartment for rent with additional information, such as price, address, area, number of beds, etc
Venues for each Manhattan neighborhood ( than can be clustered)
Venues for subway metro stations, as needed
     
The data will be used as follows:

Use Foursquare and geopy data to map top 10 venues for all Manhattan neighborhoods and clustered in groups ( as per Course LAB)
Use foursquare and geopy data to map the location of subway metro stations, separately and on top of the above clustered map in order to be able to identify the venues and amenities near each metro station, or explore each subway location separately
Use Foursquare and geopy data to map the location of rental places, in some form, linked to the subway locations.
create a map that depicts, for instance, the average rental price per square ft, around a radious of 1.0 mile (1.6 km) around each subway station - or a similar metrics. I will be able to quickly point to the popups to know the relative price per subway area.
Addresses from rental locations will be converted to geodata( lat, long) using Geopy-distance and Nominatim.
Data will be searched in open data sources if available, from real estate sites if open to reading, libraries or other government agencies such as Metro New York MTA, etc.

The processing of these DATA will allow to answer the below questions to make a decision:

what is the cost of rent (per square ft) around a mile radius from each subway metro station?
what is the area of Manhattan with best rental pricing that meets criteria established?
What is the distance from work place ( Park Ave and 53 rd St) and the tentative future home?
What are the venues of the two best places to live? How the prices compare?
How venues distribute among Manhattan neighborhoods and around metro stations?
Are there tradeoffs between size and price and location?
Any other interesting statistical data findings of the real estate and overall data.
Reference of venues around current residence in Chennai for comparison to Manhattan place.

#### Reference of venues around current residence in Chennai for comparison to Manhattan

In [8]:
# Velachery, Chennai, India
address = 'Velachery, Chennai, India'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chennai Home are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Chennai Home are 12.980165450000001, 80.22285056225584.


In [9]:
neighborhood_latitude=12.9850126
neighborhood_longitude=80.2208713

### Finding the places around the current residence in Chennai

In [10]:
CLIENT_ID = 'FZHRWCWRBO4DH0YDMEKI0JEZWRJVC4LOAAHIKBQQBFC5XKH2' # your Foursquare ID
CLIENT_SECRET = 'FZ4HND4CQDGDOO3OOOBWPMUK1JBY1C3SXBVT5OFWQYHMJPA4' # your Foursquare Secret
ACCESS_TOKEN = '4J20EAAMNFCDNPGQTBP0G0SPLOZ1C4WNTX0M1RRPBTC32MYJ' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FZHRWCWRBO4DH0YDMEKI0JEZWRJVC4LOAAHIKBQQBFC5XKH2
CLIENT_SECRET:FZ4HND4CQDGDOO3OOOBWPMUK1JBY1C3SXBVT5OFWQYHMJPA4


In [11]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=FZHRWCWRBO4DH0YDMEKI0JEZWRJVC4LOAAHIKBQQBFC5XKH2&client_secret=FZ4HND4CQDGDOO3OOOBWPMUK1JBY1C3SXBVT5OFWQYHMJPA4&v=20180604&ll=12.9850126,80.2208713&radius=500&limit=100'

In [12]:
# Send the GET Request and examine the results
results = requests.get(url).json()

###### to pull the Venue category - taken from the Foursquare lab-

In [13]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [15]:
venues = results['response']['groups'][0]['items']
SGnearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]
# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)
# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.shape

C:\Users\ramak\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


(18, 4)

In [16]:
SGnearby_venues.head(10)

,name,categories,lat,lng
0,The Westin,Hotel,12.989379,80.220885
1,Andhi Kadai,Indian Restaurant,12.982532,80.224662
2,Subway,Sandwich Place,12.987449,80.218362
3,Subway - Velachery,Sandwich Place,12.987412,80.218464
4,Peaches,Vegetarian / Vegan Restaurant,12.985401,80.218026
5,Pizza Empire,Pizza Place,12.984231,80.217984
6,Mast Kalandar,Indian Restaurant,12.984715,80.217821
7,Coal Barbecue,BBQ Joint,12.986020,80.218074
8,Pazhamudhir Cholai,Market,12.984232,80.221835
9,Dhandeeshwaram,Bus Stop,12.982931,80.222244


###### Reference - Map of Chemmai residence place with venues in Neighborhood

In [17]:
latitude=12.9850126
longitude=80.2208713
# create map of Chennai place  using latitude and longitude values
map_sg = folium.Map(location=[latitude, longitude], zoom_start=18)
# add markers to map
for lat, lng, label in zip(SGnearby_venues['lat'], SGnearby_venues['lng'], SGnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=30,
        radius=7,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.6,
    ).add_to(map_sg)  
    
map_sg

In [19]:
# Methodology, # Results, # Discussion, and # Conclusion will be discussed in the next week Project submission.